# Basic EM workflow 3 (Restaurants data set)

# Introduction

This IPython notebook explains a basic workflow two tables using *py_entitymatching*. Our goal is to come up with a workflow to match restaurants from Fodors and Zagat sites. Specifically, we want to maximize F1. The datasets contain information about the restaurants.

First, we need to import *py_entitymatching* package and other libraries as follows:

In [1]:
import sys
import py_entitymatching as em
import pandas as pd
import os

In [2]:
# Display the versions
print('python version: ' + sys.version )
print('pandas version: ' + pd.__version__ )
print('magellan version: ' + em.__version__ )

python version: 3.6.5 (default, Apr  1 2018, 05:46:30) 
[GCC 7.3.0]
pandas version: 0.22.0
magellan version: 0.3.0


# Read input tables

We begin by loading the input tables. For the purpose of this guide, we use the datasets that are included with the package.

In [3]:
# Get the paths
path_A = 'data/amazon_products.csv'
path_B = 'data/walmart_products.csv'

In [4]:
# Load csv files as dataframes and set the key attribute in the dataframe
A = em.read_csv_metadata(path_A, key='id')
B = em.read_csv_metadata(path_B, key='id')

In [5]:
print('Number of tuples in A    : ' + str(len(A)))
print('Number of tuples in B    : ' + str(len(B)))
print('Number of tuples in A X B: ' + str(len(A)*len(B)))

Number of tuples in A    : 2976
Number of tuples in B    : 4847
Number of tuples in A X B: 14424672


In [6]:
A.head(3)

,id,product_title,brand,model,operating_system,extended_title
0,a0,acer predator helios 300 15.6 full hd intel core i7 7700hq 16gb ddr4 256gb ssd geforce gtx 1060 ...,acer,nh.q28aa.001,windows 10,acer predator helios 300 15.6 full hd intel core i7 7700hq 16gb ddr4 256gb ssd geforce gtx 1060 ...
1,a1,acer aspire e 15 15.6 full hd 8th gen intel core i5 8250u geforce mx150 8gb 256gb ssd e5 576g 5762,acer,e5 576g 5762,windows 10,acer aspire e 15 15.6 full hd 8th gen intel core i5 8250u geforce mx150 8gb 256gb ssd e5 576g 57...
2,a2,asus vivobook f510ua fhd intel core i5 8250u 8gb 1tb hdd usb c nanoedge windows 10 star f510ua a...,asus,f510ua ah51,windows 10,asus vivobook f510ua fhd intel core i5 8250u 8gb 1tb hdd usb c nanoedge windows 10 star f510ua a...


In [7]:
B.head(3)

,id,product_title,brand,model,operating_system,extended_title
0,w0,iview i896qw 8.95 2 1 32gb tablet intel atom bay trail z3735f windows 10,iview,NaN,microsoft windows,iview i896qw 8.95 2 1 32gb tablet intel atom bay trail z3735f windows 10 iview microsoft
1,w1,dell inspiron 2 1 11.6 touch intel pentium 4gb 500gb hd red,dell,i3168 3270red,windows 10,dell inspiron 2 1 11.6 touch intel pentium 4gb 500gb hd i3168 3270red windows 10
2,w2,iview maximus ii 11.6 touchscreen 2 1 windows 10 intel bay trail z3735f 2gb 32gb storage,iview,max2 bk,windows 10,iview maximus ii 11.6 touchscreen 2 1 windows 10 intel bay trail z3735f 2gb 32gb iview max2 bk


In [8]:
# Display the keys of the input tables
em.get_key(A), em.get_key(B)

('id', 'id')

# Block tables to get candidate set

Before we do the matching, we would like to remove the obviously non-matching tuple pairs from the input tables. This would reduce the number of tuple pairs considered for matching.
*py_entitymatching* provides four different blockers: (1) attribute equivalence, (2) overlap, (3) rule-based, and (4) black-box. The user can mix and match these blockers to form a blocking sequence applied to input tables.

For the matching problem at hand, we know that two restaurants with different city names will not match. So we decide the apply blocking over names:

In [9]:
# Blocking plan

# A, B -- attribute equiv. blocker [model] --------------------|---> candidate set

In [10]:
# # Create overlap blocker
# ob = em.OverlapBlocker()

# # Use block_tables to apply blocking over two input tables.
# C1 = ob.block_tables(A,B, 'model', 'model', 
#                     l_output_attrs=['id','model','extended_title'], 
#                     r_output_attrs=['id','model','extended_title'],
#                     overlap_size=8,
#                     q_val=5,
#                     word_level=False,
#                     show_progress=False,
#                     n_jobs=-1
#                     )
# len(C1)

In [11]:
# C1.head(3)

## Debug blocker output

The number of tuple pairs considered for matching is reduced to 10165 (from 176423), but we would want to make sure that the blocker did not drop any potential matches. We could debug the blocker output in *py_entitymatching* as follows:

In [12]:
# Debug blocker output
# dbg = em.debug_blocker(C1, A, B, output_size=10, 
#                        attr_corres=[
#                            ('product_title','product_title'),
#                            ('brand', 'brand'), 
#                            ('model','model'),
#                            ('extended_title','extended_title')],
#                       verbose=True)
#### Display first few tuple pairs from the debug_blocker's output
# dbg.head(3)

From the debug blocker's output we observe that the current blocker drops quite a few potential matches. We would want to update the blocking sequence to avoid dropping these potential matches.

For the considered dataset, we know that for the restaurants to match the  names must overlap between them. We could use overlap blocker for this purpose. Finally, we would want to union the outputs from the attribute equivalence blocker and the overlap blocker to get a consolidated candidate set.

In [13]:
# # Create overlap blocker
# ob = em.OverlapBlocker()

# # Block tables using 'extended_title' attribute 
# C2 = ob.block_tables(A, B, 'extended_title', 'extended_title', 
#                     l_output_attrs=['id','model','extended_title'], 
#                     r_output_attrs=['id','model','extended_title'],
#                     overlap_size=25,
#                     q_val=5,
#                     word_level=False,
#                     show_progress=False,
#                     n_jobs=-1
#                     )
# len(C2)

In [14]:
# Updated blocking sequence
# A, B ------ overlap blocker [extended_title] --------> C1--|
#                                                   |----> C
# A, B ------ overlap blocker [model] --------> C2--|

In [15]:
# def match_extended_title(attribute='extended_title',q_val=3, threshold=.5,debug=False):
#     def jaccard_matcher(ltuple,rtuple, attribute=attribute, q_val=q_val, threshold=threshold,debug=debug):
#         buffer = '#' * (q_val-1)
#         l_attribute = buffer + ltuple[attribute] + buffer
#         r_attribute = buffer + rtuple[attribute] + buffer
#         l_grams = set()
#         r_grams = set()
#         # create sets of grams
#         for attribute, grams in [(l_attribute,l_grams), (r_attribute,r_grams)]:
#             for i in range(0,len(attribute)-(q_val-1)):
#                 grams.add(attribute[i:i+q_val])
                
#         # compute jaccard
#         intersection = list(set(l_grams) & set(r_grams))
#         union = list(set(l_grams) | set(r_grams))
#         if debug:
#             print(union)
#             print(intersection)
#             print(len(intersection) / len(union))
#         return len(intersection) / len(union) < threshold
        
#     return jaccard_matcher


In [16]:
# bb = em.BlackBoxBlocker()
# bb.set_black_box_function(match_extended_title())
# C3 = bb.block_candset(C2,  
#                     show_progress=False,
#                     n_jobs=-1
#                     )
# len(C3)

In [17]:
# C3.head(3)

In [18]:
# Updated blocking sequence
# A, B --- overlap blocker [model] ---> C1--------------------------------|
#                                                                   union |---> C
# A, B --- overlap blocker [extended_title] ---> C2---> jaccard blocker [extended_title] ---|

In [19]:
# # Combine blocker outputs
# C = em.combine_blocker_outputs_via_union([C1,C3])
# len(C)

In [20]:
# C.head(3)

We observe that the number of tuple pairs considered for matching is increased to 12530 (from 10165). Now let us debug the blocker output again to check if the current blocker sequence is dropping any potential matches.

In [21]:
# Debug again
# dbg = em.debug_blocker(C, A, B, output_size=100)
# dbg.head()

In [22]:
# dbg

We observe that the current blocker sequence does not drop obvious potential matches, and we can proceed with the matching step now. A subtle point to note here is, debugging blocker output practically provides a stopping criteria for modifying the blocker sequence.


# Matching tuple pairs in the candidate set

In this step, we would want to match the tuple pairs in the candidate set. Specifically, we use learning-based method for matching purposes.
This typically involves the following five steps:
1. Sampling and labeling the candidate set
2. Splitting the labeled data into development and evaluation set
3. Selecting the best learning based matcher using the development set
4. Evaluating the selected matcher using the evaluation set

## Sampling and labeling the candidate set

First, we randomly sample 450 tuple pairs for labeling purposes.

In [23]:
# # Sample  candidate set
# S = em.sample_table(C, 450)
# # em.to_csv_metadata(S, 'data/labeled.csv')

For the purposes of this guide, we will load in a pre-labeled dataset (of 450 tuple pairs) included in this package.

In [24]:
# # Load the pre-labeled data
path_S = 'data/labeled.csv'
S = em.read_csv_metadata(path_S, 
                         key='_id',
                         ltable=A, rtable=B, 
                         fk_ltable='ltable_id', fk_rtable='rtable_id')
len(S)

450

## Splitting the labeled data into development and evaluation set

In this step, we split the labeled data into two sets: development (I) and evaluation (J). Specifically, the development set is used to come up with the best learning-based matcher and the evaluation set used to evaluate the selected matcher on unseen data.

In [25]:
# Split S into development set (I) and evaluation set (J)
IJ = em.split_train_test(S, train_proportion=0.7, random_state=0)
I = IJ['train']
J = IJ['test']

## Selecting the best learning-based matcher 

Selecting the best learning-based matcher typically involves the following steps:

1. Creating a set of learning-based matchers
2. Creating features
3. Converting the development set into feature vectors
4. Selecting the best learning-based matcher using k-fold cross validation

### Creating a set of learning-based matchers

In [26]:
# Create a set of ML-matchers
dt = em.DTMatcher(name='DecisionTree', random_state=0)
svm = em.SVMMatcher(name='SVM', random_state=0)
rf = em.RFMatcher(name='RF', random_state=0)
lg = em.LogRegMatcher(name='LogReg', random_state=0)
ln = em.LinRegMatcher(name='LinReg')
nb = em.NBMatcher(name='NaiveBayes')

### Creating features

Next, we need to create a set of features for the development set. *py_entitymatching* provides a way to automatically generate features based on the attributes in the input tables. For the purposes of this guide, we use the automatically generated features.

In [27]:
# Remove bad features from auto-feature-generation
AA = A.drop(['id','product_title','model','operating_system'],axis=1)
BB = B.drop(['id','product_title','model','operating_system'],axis=1)

AA.keys(), BB.keys()

(Index(['brand', 'extended_title'], dtype='object'),
 Index(['brand', 'extended_title'], dtype='object'))

In [28]:
match_t = em.get_tokenizers_for_matching([3,5,10])
match_s = em.get_sim_funs_for_matching()
atypes1 = em.get_attr_types(AA)
atypes2 = em.get_attr_types(BB)
match_c = em.get_attr_corres(AA, BB)
feature_table = em.get_features(AA, BB, atypes1, atypes2, match_c, match_t, match_s)

In [29]:
feature_table['feature_name']

0                          brand_brand_jac_qgm_3_qgm_3
1                      brand_brand_cos_dlm_dc0_dlm_dc0
2                      brand_brand_jac_dlm_dc0_dlm_dc0
3                                      brand_brand_mel
4                                 brand_brand_lev_dist
5                                  brand_brand_lev_sim
6                                      brand_brand_nmw
7                                       brand_brand_sw
8        extended_title_extended_title_jac_qgm_3_qgm_3
9    extended_title_extended_title_cos_dlm_dc0_dlm_dc0
Name: feature_name, dtype: object

This function generates a function that returns 1 if both or neither tuples' attribute contain any of the passed in values and 0 otherwise.

In [30]:
def generateContainsValueFeature(values, name=None, attribute='extended_title'):
    if type(values) is str:
        values = [values]
    def containsValueFeature(a,b):
        return int(any([value.lower() in a[attribute].lower() for value in values]) 
                   == any([value.lower() in b[attribute].lower() for value in values]))
    return containsValueFeature, name if name else values[0]

Use this to generate many new features.

In [31]:
brands = ['lg','toshiba','hp','dell','lenovo','prostar','acer','samsung','apple','asus','panasonic','msi']
models = ['zephyrus','zenbook','flex','omen','xps','x1','carbon','yoga',
          'latitude','inspiron','elitebook','clevo','spectre',
          'macbook','pavilion','ideapad','legion']
thinkpads = [['p40','p50','p51','p71'],['430','460','470','560','570']]
asus = ['swift','aspire','spin']
models = models + thinkpads + asus
sizes = [' 13',' 14',' 15',' 17']
operating_systems = ['chrome','windows','mac']
cpus = [['i3','i5','i7'],['celeron','pentium'],['m3','m5'],'amd']
miscellaneous = ['2-in-1','gtx','touch']
keywords = models \
    + sizes  \
    + operating_systems \
    + cpus \
    + miscellaneous
new_features = [generateContainsValueFeature(value) for value in keywords]

for feature in new_features:
    em.add_blackbox_feature(feature_table, feature[1], feature[0])

This function generates a function that returns 1 if both tuples contain the same value for any of the values passed in and 0 otherwise.

In [32]:
def generateContainsValueFromValuesetFeature(valueset, name=None, attribute='extended_title'):
    def sharesValue(a,b,values):
        if type(values) is str:
            values = [values]
        return int(any([value.lower() in a[attribute].lower() and value.lower() in b[attribute].lower() for value in values]))
    def containsValueFromValueset(a,b):
        return any([sharesValue(a,b,values) for values in valueset[1]])
    return containsValueFromValueset, valueset[0]

In [33]:
valuesets = [('brands',brands), 
             ('models', models), 
             ('sizes', sizes), 
             ('cpus', cpus), 
             ('operating_systems',operating_systems)]

new_features = [generateContainsValueFromValuesetFeature(valueset) for valueset in valuesets]

for feature in new_features:
    em.add_blackbox_feature(feature_table, feature[1], feature[0])

In [34]:
# List the names of the features generated
feature_table['feature_name']

0                           brand_brand_jac_qgm_3_qgm_3
1                       brand_brand_cos_dlm_dc0_dlm_dc0
2                       brand_brand_jac_dlm_dc0_dlm_dc0
3                                       brand_brand_mel
4                                  brand_brand_lev_dist
5                                   brand_brand_lev_sim
6                                       brand_brand_nmw
7                                        brand_brand_sw
8         extended_title_extended_title_jac_qgm_3_qgm_3
9     extended_title_extended_title_cos_dlm_dc0_dlm_dc0
10                                             zephyrus
11                                              zenbook
12                                                 flex
13                                                 omen
14                                                  xps
15                                                   x1
16                                               carbon
17                                              

### Converting the development set to feature vectors

In [35]:
# Convert the I into a set of feature vectors using F
H = em.extract_feature_vecs(I, 
                            feature_table=feature_table, 
                            attrs_after='match',
                            show_progress=False)
H.fillna(0, inplace=True)
H.head()

,_id,ltable_id,rtable_id,brand_brand_jac_qgm_3_qgm_3,brand_brand_cos_dlm_dc0_dlm_dc0,brand_brand_jac_dlm_dc0_dlm_dc0,brand_brand_mel,brand_brand_lev_dist,brand_brand_lev_sim,brand_brand_nmw,...,amd,2-in-1,gtx,touch,brands,models,sizes,cpus,operating_systems,match
221,3372,a1997,w2968,1.00,1.0,1.0,1.00000,0.0,1.0000,6.0,...,1,1,1,0,True,True,True,True,True,1
439,6763,a891,w2599,0.04,0.0,0.0,0.60625,13.0,0.1875,-7.0,...,1,1,1,1,True,False,False,True,True,0
191,2875,a1919,w3250,1.00,1.0,1.0,1.00000,0.0,1.0000,6.0,...,1,1,1,1,True,True,True,True,True,1
239,3626,a1997,w3468,1.00,1.0,1.0,1.00000,0.0,1.0000,6.0,...,1,1,1,1,True,True,True,True,True,1
433,6651,a878,w2294,1.00,1.0,1.0,1.00000,0.0,1.0000,6.0,...,1,1,1,1,True,True,True,True,True,1


### Selecting the best matcher using cross-validation

Now, we select the best matcher using k-fold cross-validation. For the purposes of this guide, we use five fold cross validation and use the 'precision' metric to select the best matcher.

In [36]:
# Select the best ML matcher using CV
result = em.select_matcher(
        matchers=[dt, rf, svm, ln, lg, nb], 
        table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', 
        metric_to_select_matcher='precision', 
        random_state=0)
result['cv_stats']
# result

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.954048,0.937980,0.945281
1,RF,0.963564,0.972000,0.967336
2,SVM,0.893285,0.992000,0.939676
3,LinReg,0.962487,0.952694,0.957178
4,LogReg,0.913782,0.960000,0.935088
5,NaiveBayes,0.946487,0.942230,0.943907


### Debugging matcher

We observe that the best matcher is not maximizing F1. We debug the matcher to see what might be wrong.
To do this, first we split the feature vectors into train and test.

In [37]:
#  Split feature vectors into train and test
UV = em.split_train_test(H, train_proportion=0.5)
U = UV['train']
V = UV['test']

Next, we debug the matcher using GUI. For the purposes of this guide, we use random forest matcher for debugging purposes.

In [38]:
# Debug rf using GUI
# em.vis_debug_dt(result['selected_matcher'], U, V, 
#         exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
#         target_attr='match')

##  Evaluating the matching output

From the GUI, we observe that phone numbers seem to be an important attribute, but they are in different format. Current features does not capture and adding a feature incorporating this difference in format can potentially improve 
the F1 numbers.

Now, we repeat extracting feature vectors (this time with updated feature table), imputing table and selecting the best matcher again using cross-validation.

In [39]:
# Select the best ML matcher using CV
result = em.select_matcher(
        [dt, rf, svm, ln, lg, nb], 
        table=H, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
        k=5,
        target_attr='match', 
        metric_to_select_matcher='precision', 
        random_state=0)
result['cv_stats']

,Matcher,Average precision,Average recall,Average f1
0,DecisionTree,0.954048,0.937980,0.945281
1,RF,0.963564,0.972000,0.967336
2,SVM,0.893285,0.992000,0.939676
3,LinReg,0.962487,0.952694,0.957178
4,LogReg,0.913782,0.960000,0.935088
5,NaiveBayes,0.946487,0.942230,0.943907


Evaluating the matching outputs for the evaluation set typically involves the following four steps:
1. Converting the evaluation set to feature vectors
2. Training matcher using the feature vectors extracted from the development set
3. Predicting the evaluation set using the trained matcher
4. Evaluating the predicted matches

### Converting the evaluation set to  feature vectors

As before, we convert to the feature vectors (using the feature table and the evaluation set)

In [40]:
# Convert J into a set of feature vectors using feature table
L = em.extract_feature_vecs(
        J, 
        feature_table=feature_table,
        attrs_after='match', 
        show_progress=False)
L.fillna(0, inplace=True)
L.head()

,_id,ltable_id,rtable_id,brand_brand_jac_qgm_3_qgm_3,brand_brand_cos_dlm_dc0_dlm_dc0,brand_brand_jac_dlm_dc0_dlm_dc0,brand_brand_mel,brand_brand_lev_dist,brand_brand_lev_sim,brand_brand_nmw,...,amd,2-in-1,gtx,touch,brands,models,sizes,cpus,operating_systems,match
124,1764,a1561,w15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1,1,1,1,True,True,False,True,True,0
54,705,a1269,w3092,1.0,1.0,1.0,1.0,0.0,1.0,6.0,...,1,1,1,0,True,True,True,True,True,1
268,4200,a2294,w1544,1.0,1.0,1.0,1.0,0.0,1.0,7.0,...,1,1,1,1,True,True,True,True,True,1
293,4614,a2439,w3188,1.0,1.0,1.0,1.0,0.0,1.0,6.0,...,1,1,1,1,True,True,True,True,True,1
230,3469,a1997,w3210,1.0,1.0,1.0,1.0,0.0,1.0,6.0,...,1,1,1,1,True,True,True,True,True,1


### Training the selected matcher

Now, we train the matcher using all of the feature vectors from the development set. For the purposes of this guide we use random forest as the selected matcher.

In [41]:
# Train using feature vectors from I 
result['selected_matcher'].fit(table=H, 
       exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
       target_attr='match')

### Predicting the matches

Next, we predict the matches for the evaluation set (using the feature vectors extracted from it).

In [42]:
# Predict on L 
predictions = result['selected_matcher'].predict(
        table=L, 
        exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'], 
        append=True, 
        target_attr='predicted', 
        inplace=False)

### Evaluating the predictions

Finally, we evaluate the accuracy of predicted outputs

In [43]:
# Evaluate the predictions
eval_result = em.eval_matches(predictions, 'match', 'predicted')
em.print_eval_summary(eval_result)

Precision : 94.51% (86/91)
Recall : 92.47% (86/93)
F1 : 93.48%
False positives : 5 (out of 91 positive predictions)
False negatives : 7 (out of 44 negative predictions)


In [44]:
# em.vis_debug_dt(dt, 
#                 U, V, 
#                 exclude_attrs=['_id', 'ltable_id', 'rtable_id', 'match'],
#                 target_attr='match')